In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os
from functools import reduce
from itertools import chain
import pandas as pd
import json
from tqdm import tqdm
from pprint import pprint
from collections import defaultdict
from ema.utils import get_names_synonyms_doid, get_doid_names, clean_text, contains_word
import pickle
from wikidataintegrator import wdi_helpers, wdi_core
import unicodedata
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 100)

site uses ridiculous coldfusion shit which Im not going to figure out, so go this url  
http://www.accessdata.fda.gov/scripts/opdlisting/oopd/index.cfm

change records per page to 9999, hit run search  
save page as page.html

In [2]:
df = pd.read_html("page.html", header=0, index_col=None)[2]
del df['#']
df = df.loc[df['Generic Name'].notnull(), :]
len(df)

4015

In [3]:
df.head()

,Generic Name,Orphan Designation,Designation Date,Designation Status
0,"(+/-)-7-[3-(4-acetyl-3-methoxy-2-propylphenoxy)propoxy]-3,4-dihydro-8-propyl-2H-1-benzopyran-2-carboxylic acid",Prevention of serious adverse events associated with vascular leak syndrome caused by Interleukin-2 therapy,03/31/2003,Designated/Withdrawn
1,"(+/-)-cis-3-(4-hydroxyphenyl)-4-(4-methoxyphenyl)-3,4-dihydro-2H-cromen-7-ol",Treatment of Stage IIB through Stage IV malignant melanoma,02/01/2008,Designated/Withdrawn
2,"(+/-)-cis-3-(4-hydroxyphenyl)-4-(4-methoxyphenyl)-3,4-dihydro-2H-cromen-7-ol",Treatment of cholangiocarcinoma,01/10/2008,Designated/Withdrawn
3,"(+/-)-cis-3-(4-hydroxyphenyl)-4-(4-methoxyphenyl)-3,4-dihydro-2H-cromen-7-ol",Treatment of pancreatic cancer,01/10/2008,Designated/Withdrawn
4,"(-)-(3aR,4S,7aR)-4-Hydroxy-4-m-tolylethynyl-octahydro-indole-1-carboxylic acid methyl ester",Treatment of Fragile X syndrome,10/12/2011,Designated


In [4]:
products = list(set(list(df['Generic Name'])))
len(products)

3148

In [5]:
# use OPSIN to convert IUPAC names to InChI keys https://bitbucket.org/dan2097/opsin/
with open('input.txt', 'w') as f:
    for x in products:
        f.write(x + '\n')

In [6]:
!java -jar ../data/opsin-2.2.0-jar-with-dependencies.jar -ostdinchikey input.txt output.txt 2> output.err

In [7]:
prod_inchi = dict()
with open('output.txt', 'r') as f:
    for count, x in enumerate(f.readlines()):
        ikey = x.strip()
        if ikey:
            prod_inchi[products[count]] = ikey

In [8]:
len(prod_inchi)

261

In [9]:
inchi_wd = wdi_helpers.id_mapper("P235")
wd_inchi = {v:k for k,v in inchi_wd.items()}

In [10]:
# map inchikey back to wdid
prod_wdid = {prod: inchi_wd[inchi] for prod,inchi in prod_inchi.items() if inchi in inchi_wd}
len(prod_wdid)

158

In [11]:
drug_qid_map = pickle.load(open("../data/drug_qid_map.pkl", 'rb'))
#drug_qid_map = get_drug_qid_map()
wdid_search_name = {x:drug_qid_map[x.lower()] for x in products if x.lower() in drug_qid_map}

In [12]:
len(wdid_search_name)

1061

In [13]:
prod_wdid.update(wdid_search_name)

In [14]:
df['drug_qid'] = df['Generic Name'].apply(prod_wdid.get)
df['inchi'] = df['Generic Name'].apply(prod_inchi.get)

In [15]:
df

,Generic Name,Orphan Designation,Designation Date,Designation Status,drug_qid,inchi
0,"(+/-)-7-[3-(4-acetyl-3-methoxy-2-propylphenoxy)propoxy]-3,4-dihydro-8-propyl-2H-1-benzopyran-2-carboxylic acid",Prevention of serious adverse events associated with vascular leak syndrome caused by Interleukin-2 therapy,03/31/2003,Designated/Withdrawn,Q27088744,ZVVCSBSDFGYRCB-UHFFFAOYSA-N
1,"(+/-)-cis-3-(4-hydroxyphenyl)-4-(4-methoxyphenyl)-3,4-dihydro-2H-cromen-7-ol",Treatment of Stage IIB through Stage IV malignant melanoma,02/01/2008,Designated/Withdrawn,None,None
2,"(+/-)-cis-3-(4-hydroxyphenyl)-4-(4-methoxyphenyl)-3,4-dihydro-2H-cromen-7-ol",Treatment of cholangiocarcinoma,01/10/2008,Designated/Withdrawn,None,None
3,"(+/-)-cis-3-(4-hydroxyphenyl)-4-(4-methoxyphenyl)-3,4-dihydro-2H-cromen-7-ol",Treatment of pancreatic cancer,01/10/2008,Designated/Withdrawn,None,None
4,"(-)-(3aR,4S,7aR)-4-Hydroxy-4-m-tolylethynyl-octahydro-indole-1-carboxylic acid methyl ester",Treatment of Fragile X syndrome,10/12/2011,Designated,Q6589598,ZFPZEYHRWGMJCV-ZHALLVOQSA-N
5,"(1,3)-ß-D-glucan synthesis inhibitor",Treatment of invasive Aspergillus infections,08/22/2016,Designated,None,None
6,"(1-methyl-2-nitro-1H-imidazole-5-yl)methyl N,N'-bis(2-broethyl) diamidophosphate",Treatment of pancreatic cancer,06/05/2013,Designated,None,None
7,"(1-methyl-2-nitro-1H-imidazole-5-yl)methyl N,N'-bis(2-bromoethyl) diamidophosphate",Treatment of soft tissue sarcoma,03/09/2012,Designated,None,None
8,"(1OR)-7-amino-12-fluoro-2,10,16-trimethyl-15 oxo-10,15,16,17-tetrahydro-2H-8,4-(metheno)pyrazolo[4,3-h][2,5,11]benzoxadiazacyclotetradecine-3-carbonitrile",Treatment of anaplastic lymphoma kinase (ALK)-positive or ROS1-positive non-small cell lung cancer,06/23/2015,Designated,None,None
9,"(1R,3R,4R,5S)-3-O-[2-O-benzoyl-3-O-(sodium(2S)-3-cyclohexyl-propanoate-",Treatment of vaso-occlusive crisis in patients with sickle cell disease.,02/17/2009,Designated,None,None


In [16]:
# https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=0ahUKEwixz4Knh6PSAhVjrFQKHWr0C-QQFggaMAA&url=http%3A%2F%2Fwww.fda.gov%2Fdownloads%2FForIndustry%2F...%2FHowtoapplyforOrphanProductDesignation%2FUCM162066.xls&usg=AFQjCNFnMUzkWwsJCPnmqzabQeAIUOicVQ&sig2=OD5moBW6BuDdi6K2PGag1A&bvm=bv.147448319,d.cGw

In [17]:
df.count()

Generic Name          4015
Orphan Designation    4015
Designation Date      4015
Designation Status    4015
drug_qid              1753
inchi                 323 
dtype: int64

## Indications

In [18]:
first_word = df['Orphan Designation'].str.split(" ").apply(lambda x:x[0].lower())
first_word.value_counts().head(10)

treatment      3306
for            254 
prevention     164 
prophylaxis    29  
to             28  
management     20  
diagnostic     17  
1.             15  
as             12  
use            11  
Name: Orphan Designation, dtype: int64

In [19]:
first_word_idx = first_word.isin({'treatment', 'prevention', 'diagnosis', 'prophylaxis', 'management', 'diagnostic'})
df.loc[first_word_idx, 'indication'] = df.loc[first_word_idx, 'Orphan Designation'].str.split(" ").apply(lambda x: " ".join(x[2:]).lower())
df.loc[~first_word_idx, 'indication'] = df.loc[~first_word_idx, 'Orphan Designation']
df.indication = df.indication.str.lower()
df.indication = df.indication.dropna().apply(lambda x:x[:-1] if x.endswith('.') else x)
diseases = set(list(df['indication'].dropna().str.strip())) - {""}
len(diseases)

2315

In [20]:
## orphanet
#http://data.bioontology.org/ontologies/ORDO/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb&download_format=csv
ordo = pd.read_csv("../data/ORDO.csv.gz")
ordo.dropna(subset=['Preferred Label'], inplace=True)
name_ordo = dict(zip(ordo['Preferred Label'].str.lower(), ordo['Class ID']))
name_ordo = {k: v for k,v in name_ordo.items()}
for _, row in ordo[ordo.Synonyms.notnull()].iterrows():
    for syn in row.Synonyms.split("|"):
        name_ordo[syn.lower()] = row['Class ID']
del name_ordo['disease']
ordo_matches = {x:name_ordo[x] for x in diseases if x in name_ordo}
len(ordo_matches)

262

In [21]:
# match to DO or HPO
def onto_synonyms(json_path):
    doid = json.loads(open(json_path).read())
    graph = doid['graphs'][0]
    nodes = graph['nodes']
    name_doid = dict()
    for node in nodes:
        if 'meta' in node and 'deprecated' in node['meta'] and node['meta']['deprecated']:
            continue
        if 'lbl' not in node:
            continue
        name_doid[node['lbl'].lower()] = node['id']
        if 'meta' in node and 'synonyms' in node['meta']:
            for syn in node['meta']['synonyms']:
                name_doid[syn['val'].lower()] = node['id']
    return name_doid
name_doid = onto_synonyms("../data/doid.json")
name_hpo = onto_synonyms("../data/hp.json")
print(len(name_doid))
print(len(name_hpo))

21114
26710


In [22]:
doid_matches = {x:name_doid[x] for x in diseases if x in name_doid}
len(doid_matches)

297

In [23]:
hpo_matches = {x:name_hpo[x] for x in diseases if x in name_hpo}
len(hpo_matches)

124

In [24]:
# create mesh title synonyms dict
# json is created from https://github.com/stuppie/mesh-parser
mesh = json.load(open("/home/gstupp/projects/mesh/mesh.json"))
name_mesh = dict()
for record in mesh.values():
    if record['record_type'] == 'C':
        continue
    name_mesh[record['term'].lower()] = record['_id']
    for syn in record.get('synonyms', []):
        name_mesh[syn.lower()] = record['_id']
mesh_name = {v:k for k,v in name_mesh.items()}

In [25]:
mesh_matches = {x:name_mesh[x] for x in diseases if x in name_mesh}
len(mesh_matches)

376

In [26]:
diseases_not_matched = {x for x in diseases if x not in doid_matches and x not in ordo_matches and x not in mesh_matches and x not in hpo_matches}
len(diseases_not_matched)

1847

In [27]:
diseases_not_matched

{'for the treatment of graft versus host disease',
 'prophylaxis of bleeding episodes in patients with hemophilia a or b with inhibitors to factor viii or factor ix',
 'hepatitis c infection in liver transplant recipients',
 'patients at high risk of graft rejection following penetrating keratoplasty',
 'invasive candida infections, including candidemia',
 'stage ii (t4), iii or iv malignant melanoma',
 'idiopathic thrombocytopenia purpura',
 'partial onset or primary generalized tonic-clonic seizures for hospitalized epilepsy patients or epilepsy patients being treated in an emergency care setting who are unable to take oral topiramate',
 'patients with lymphatic malformations',
 'for post-exposure prophylaxis of anthrax disease resulting from suspected or confirmed bacillus anthracis exposure',
 'for localization of sentinel lymph nodes in patients with melanoma',
 'patients with known hepatocellular carcinoma (hcc)',
 'hyperuricemia in patients with gout refractory to conventional t

In [28]:
df['doid'] = df.indication.apply(doid_matches.get)
df['hpo'] = df.indication.apply(hpo_matches.get)
df['ordo'] = df.indication.apply(ordo_matches.get)
df['mesh'] = df.indication.apply(mesh_matches.get)

In [29]:
doid_wd = wdi_helpers.id_mapper("P699")
ordo_wd = wdi_helpers.id_mapper("P1550")
mesh_wd = wdi_helpers.id_mapper("P486")

In [30]:
"squamous non-small cell lung cancer"
# http://www.ebi.ac.uk/ols/ontologies/hp/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FHP_0030359

'squamous non-small cell lung cancer'

In [31]:
df.loc[:,'indication_qid_doid'] = df.doid.dropna().apply(lambda x:doid_wd.get(x.split("/")[-1].replace("_",":")))
df.loc[:,'indication_qid_ordo'] = df.ordo.dropna().apply(lambda x:ordo_wd.get(x.split("_")[-1]))
df.loc[:,'indication_qid_mesh'] = df.mesh.dropna().apply(mesh_wd.get)

In [32]:
df.head()

,Generic Name,Orphan Designation,Designation Date,Designation Status,drug_qid,inchi,indication,doid,hpo,ordo,mesh,indication_qid_doid,indication_qid_ordo,indication_qid_mesh
0,"(+/-)-7-[3-(4-acetyl-3-methoxy-2-propylphenoxy)propoxy]-3,4-dihydro-8-propyl-2H-1-benzopyran-2-carboxylic acid",Prevention of serious adverse events associated with vascular leak syndrome caused by Interleukin-2 therapy,03/31/2003,Designated/Withdrawn,Q27088744,ZVVCSBSDFGYRCB-UHFFFAOYSA-N,serious adverse events associated with vascular leak syndrome caused by interleukin-2 therapy,None,None,None,None,NaN,NaN,NaN
1,"(+/-)-cis-3-(4-hydroxyphenyl)-4-(4-methoxyphenyl)-3,4-dihydro-2H-cromen-7-ol",Treatment of Stage IIB through Stage IV malignant melanoma,02/01/2008,Designated/Withdrawn,None,None,stage iib through stage iv malignant melanoma,None,None,None,None,NaN,NaN,NaN
2,"(+/-)-cis-3-(4-hydroxyphenyl)-4-(4-methoxyphenyl)-3,4-dihydro-2H-cromen-7-ol",Treatment of cholangiocarcinoma,01/10/2008,Designated/Withdrawn,None,None,cholangiocarcinoma,http://purl.obolibrary.org/obo/DOID_4947,http://purl.obolibrary.org/obo/HP_0030153,http://www.orpha.net/ORDO/Orphanet_70567,D018281,Q124292,None,Q18556554
3,"(+/-)-cis-3-(4-hydroxyphenyl)-4-(4-methoxyphenyl)-3,4-dihydro-2H-cromen-7-ol",Treatment of pancreatic cancer,01/10/2008,Designated/Withdrawn,None,None,pancreatic cancer,http://purl.obolibrary.org/obo/DOID_1793,http://purl.obolibrary.org/obo/HP_0002894,None,D010190,Q212961,NaN,Q212961
4,"(-)-(3aR,4S,7aR)-4-Hydroxy-4-m-tolylethynyl-octahydro-indole-1-carboxylic acid methyl ester",Treatment of Fragile X syndrome,10/12/2011,Designated,Q6589598,ZFPZEYHRWGMJCV-ZHALLVOQSA-N,fragile x syndrome,http://purl.obolibrary.org/obo/DOID_14261,None,http://www.orpha.net/ORDO/Orphanet_908,D005600,Q221472,Q221472,Q221472


In [33]:
df.to_csv("fda_orphan_match.csv")

In [34]:
good_df=df[df.indication_qid_doid.notnull() & df.drug_qid.notnull()]
good_df

,Generic Name,Orphan Designation,Designation Date,Designation Status,drug_qid,inchi,indication,doid,hpo,ordo,mesh,indication_qid_doid,indication_qid_ordo,indication_qid_mesh
4,"(-)-(3aR,4S,7aR)-4-Hydroxy-4-m-tolylethynyl-octahydro-indole-1-carboxylic acid methyl ester",Treatment of Fragile X syndrome,10/12/2011,Designated,Q6589598,ZFPZEYHRWGMJCV-ZHALLVOQSA-N,fragile x syndrome,http://purl.obolibrary.org/obo/DOID_14261,None,http://www.orpha.net/ORDO/Orphanet_908,D005600,Q221472,Q221472,Q221472
16,"(2,2-diphenyl-tetrahydro-furan-3-yl-methyl)-dimethylamine hydrochloride",Treatment of Rett Syndrome.,05/18/2016,Designated,Q27252594,FEQOLYDPQKHFTD-UHFFFAOYSA-N,rett syndrome,http://purl.obolibrary.org/obo/DOID_1206,None,http://www.orpha.net/ORDO/Orphanet_778,D015518,Q917357,None,Q917357
17,(2-((2-chlorophenyl)(phenyl)amino)-N-(7-(hydroxyamino)-7-oxoheptyl)pyrimidine-5-carboxamide),Treatment of multiple myeloma,11/29/2016,Designated,Q27258687,VLIUIBXPEDFJRF-UHFFFAOYSA-N,multiple myeloma,http://purl.obolibrary.org/obo/DOID_9538,http://purl.obolibrary.org/obo/HP_0006775,http://www.orpha.net/ORDO/Orphanet_29073,D009101,Q467635,None,Q467635
18,"(2-(2-chlorophenyl)-4-[3-(dimethylamino)phenyl]-5-methyl-1h-pyrazolo [4,3-c] pyridine-3,6(2h,5h)-dione)",Treatment of systemic sclerosis.,10/14/2015,Designated,Q27258840,RGYQPQARIQKJKH-UHFFFAOYSA-N,systemic sclerosis,http://purl.obolibrary.org/obo/DOID_418,None,http://www.orpha.net/ORDO/Orphanet_90291,D012595,Q5340515,None,Q5340515
30,"(3S)-1-azabicyclo[2.2.2]oct-3-yl {2-[2-(4-fluorophenyl)-1,3-thiazol-4-yl]propan-2-yl}carbamate",Treatment of Gaucher disease,09/11/2014,Designated,Q27274745,YFHRCLAKZBDRHN-MRXNPFEDSA-N,gaucher disease,http://purl.obolibrary.org/obo/DOID_1926,None,http://www.orpha.net/ORDO/Orphanet_355,D005776,Q861645,Q861645,Q861645
40,"(5R)-5-(4-{[(2-fluorophenyl)methyl]oxy}phenyl)-L-prolinamide, hydrochloride",Treatment of erythromelalgia,01/30/2017,Designated,Q27268497,HEPUBAGOKRIZEO-PPPUBMIESA-N,erythromelalgia,http://purl.obolibrary.org/obo/DOID_9240,None,http://www.orpha.net/ORDO/Orphanet_1956,D004916,Q524353,None,Q524353
57,"(R)-2-methyl-6-nitro-2-{4-[4-(4-trifluoromethoxyphenoxy)piperidin-1-yl]phenoxymethyl}-2,3-dihydroimidazo[2,1-b]oxazole",Treatment of pulmonary tuberculosis.,07/12/2007,Designated,Q15408413,XDAOLTSRNUSPPH-XMMPIXPASA-N,pulmonary tuberculosis,http://purl.obolibrary.org/obo/DOID_2957,None,None,D014397,Q18555243,NaN,Q18555243
58,"(R)-4-(3-morpholin-4-yl-1-phenylsulfanylmethyl-propylamino)-N-(4-{4-[2-(4-chlorophenyl)-5,5-dimethylcyclohex-1-enylmethyl]-piperazin-1-yl}-benzoyl)-3-trifluoromethanesulfonylbenzenesulfonamide bis-hydrochloride",Treatment of multiple myeloma,05/05/2008,Designated/Withdrawn,Q27292463,WDVGRPCSLPVWKC-VROLVAQFSA-N,multiple myeloma,http://purl.obolibrary.org/obo/DOID_9538,http://purl.obolibrary.org/obo/HP_0006775,http://www.orpha.net/ORDO/Orphanet_29073,D009101,Q467635,None,Q467635
65,"(S)-1-(6,7-Dihydro-5H-benzo[6,7]cyclohepta[1,2-c]pyridazin-3-yl)-N3-(7-(pyrrolidin-1-yl)-6,7,8,9-tetrahydro-5H-benzo[7]annulen-2-yl)-1H-1,2,4-triazole-3,5-diamine",Treatment of acute myeloid leukemia,11/05/2014,Designated,Q27236818,KXMZDGSRSGHMMK-VWLOTQADSA-N,acute myeloid leukemia,http://purl.obolibrary.org/obo/DOID_9119,http://purl.obolibrary.org/obo/HP_0004808,http://www.orpha.net/ORDO/Orphanet_519,D015470,Q264118,None,Q264118
70,(S)-3-(1-(9H-purin-6-ylamino)ethyl)-8-chloro-2-phenylisoquinolin-1(2H)-one,Treatment of follicular lymphoma,08/01/2013,Designated,Q27077129,SJVQHLPISAIATJ-ZDUSSCGKSA-N,follicular lymphoma,http://purl.obolibrary.org/obo/DOID_0050873,None,http://www.orpha.net/ORDO/Orphanet_545,D008224,Q123251,None,None


In [35]:
len(good_df)

677

In [36]:
df.count()

Generic Name           4015
Orphan Designation     4015
Designation Date       4015
Designation Status     4015
drug_qid               1753
inchi                  323 
indication             4015
doid                   1490
hpo                    902 
ordo                   1294
mesh                   1735
indication_qid_doid    1490
indication_qid_ordo    286 
indication_qid_mesh    1633
dtype: int64